# TSA Chapter 2: Model Selection with AIC and BIC

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch2/TSA_ch2_model_selection/TSA_ch2_model_selection.ipynb)

This notebook demonstrates:
- AIC and BIC information criteria for ARMA model selection
- Comparison of 9 candidate models fitted to ARMA(1,1) data, complexity vs fit trade-off

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Set random seed

n = 300

print("=" * 60)
print("MODEL SELECTION: AIC AND BIC")
print("=" * 60)

print("""
Information Criteria:

AIC = -2 ln(L̂) + 2k
BIC = -2 ln(L̂) + k ln(n)

Where:
  L̂ = maximized likelihood
  k = number of parameters
  n = sample size

Key Differences:
  - AIC: penalizes complexity less → larger models
  - BIC: penalizes complexity more → more parsimonious models
  - BIC penalty grows with n → increasingly favors simpler models

Rule: LOWER is BETTER
""")

# Generate true ARMA(1,1) data
phi_true = 0.7
theta_true = 0.4
ar = np.array([1, -phi_true])
ma = np.array([1, theta_true])
arma_process = ArmaProcess(ar, ma)
data = arma_process.generate_sample(nsample=n)

# Fit various models and compare
models = [
    (1, 0, 'AR(1)'),
    (2, 0, 'AR(2)'),
    (3, 0, 'AR(3)'),
    (0, 1, 'MA(1)'),
    (0, 2, 'MA(2)'),
    (1, 1, 'ARMA(1,1)'),
    (2, 1, 'ARMA(2,1)'),
    (1, 2, 'ARMA(1,2)'),
    (2, 2, 'ARMA(2,2)')
]

results = []

print("\n" + "=" * 60)
print("FITTING MULTIPLE MODELS TO ARMA(1,1) DATA")
print("=" * 60)
print(f"True model: ARMA(1,1) with φ={phi_true}, θ={theta_true}")
print(f"Sample size: n = {n}")
print("-" * 60)
print(f"{'Model':<12} {'k':>5} {'Log-Like':>12} {'AIC':>10} {'BIC':>10}")
print("-" * 60)

for p, q, name in models:
    try:
        model = ARIMA(data, order=(p, 0, q))
        fit = model.fit()
        k = p + q + 1  # AR + MA + variance
        results.append({
            'Model': name,
            'p': p,
            'q': q,
            'k': k,
            'LogLike': fit.llf,
            'AIC': fit.aic,
            'BIC': fit.bic
        })
        print(f"{name:<12} {k:>5} {fit.llf:>12.2f} {fit.aic:>10.2f} {fit.bic:>10.2f}")
    except Exception as e:
        print(f"{name:<12} -- fitting failed")

df = pd.DataFrame(results)

# Find best models
best_aic = df.loc[df['AIC'].idxmin()]
best_bic = df.loc[df['BIC'].idxmin()]

print("-" * 60)
print(f"\nBest by AIC: {best_aic['Model']} (AIC = {best_aic['AIC']:.2f})")
print(f"Best by BIC: {best_bic['Model']} (BIC = {best_bic['BIC']:.2f})")

# Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# AIC comparison
ax1 = axes[0]
colors = ['green' if m == best_aic['Model'] else 'blue' for m in df['Model']]
ax1.barh(df['Model'], df['AIC'], color=colors, alpha=0.7, edgecolor='black')
ax1.axvline(x=best_aic['AIC'], color='green', linestyle='--', linewidth=2)
ax1.set_xlabel('AIC (lower is better)')
ax1.set_title('AIC Comparison')
ax1.grid(True, alpha=0.3, axis='x')

# BIC comparison
ax2 = axes[1]
colors = ['green' if m == best_bic['Model'] else 'blue' for m in df['Model']]
ax2.barh(df['Model'], df['BIC'], color=colors, alpha=0.7, edgecolor='black')
ax2.axvline(x=best_bic['BIC'], color='green', linestyle='--', linewidth=2)
ax2.set_xlabel('BIC (lower is better)')
ax2.set_title('BIC Comparison')
ax2.grid(True, alpha=0.3, axis='x')

# Parameter count vs fit
ax3 = axes[2]
ax3.scatter(df['k'], df['AIC'], s=100, label='AIC', alpha=0.7)
ax3.scatter(df['k'], df['BIC'], s=100, marker='s', label='BIC', alpha=0.7)
for _, row in df.iterrows():
    ax3.annotate(row['Model'], (row['k'], row['AIC']), xytext=(5, 5),
                 textcoords='offset points', fontsize=8)
ax3.set_xlabel('Number of Parameters (k)')
ax3.set_ylabel('Information Criterion')
ax3.set_title('Complexity vs Fit Trade-off')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
save_chart(fig, 'ch2_model_selection')
plt.show()

print("\n" + "=" * 60)
print("MODEL SELECTION GUIDELINES")
print("=" * 60)
print("""
1. Start with ACF/PACF to get initial p, q estimates
2. Fit candidate models (vary p and q around initial guess)
3. Compare using AIC and BIC
4. Check residual diagnostics (Ljung-Box test)
5. Use out-of-sample validation if possible

When AIC and BIC Disagree:
  - AIC tends to select larger models
  - BIC tends to select smaller models
  - BIC is consistent (selects true model as n → ∞)
  - AIC is asymptotically efficient (best predictions)

Practical Advice:
  - Use BIC for model interpretation
  - Use AIC for forecasting
  - Consider both and check residuals
""")